참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengage Learning, 2013.

`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)<br>
Bring in `module`'s that would expand features of `python`. (This file may use just some of them.)

In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import sympy.plotting as splot
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비

## 예제 05.011<br>Ex. 05.001

### 문제에서 주어진 변수<br>Given Parameters<br>Variables from Problem

#### 웨브 web 단면<br>Section of web

In [ ]:
h_web_m, t_web_m = sy.symbols('h_{web}[m], t_{web}[m]', positive=True)

s_d = {h_web_m: 1100e-3, 
       t_web_m: 10e-3}

#### 앵글 angle 단면<br>Section of angle

In [ ]:
t_angle_m, A_angle_m2, y_bar_m = sy.symbols('t_{angle}[m], A_{angle}[m^{2}], y_{bar}[m]', positive=True)
n_angle = sy.symbols('n_angle', integer=True, positive=True)

# Nick Fortescue, Jacob Marble, '''Python “extend” for a dictionary''', Stackoverflow, 2013 March 23, https://stackoverflow.com/questions/577234/python-extend-for-a-dictionary
s_d.update({t_angle_m: 13e-3,
            n_angle: 4,
            A_angle_m2: 2630 * (1e-3 ** 2),
            y_bar_m: 536.3e-3,
           })

#### 전체 단면의 중립축에 대한 I<br>I of the whole section with respect to the neutral axis

In [ ]:
I_m4 = sy.symbols('I[m^{4}]', positive=True)

# Nick Fortescue, Jacob Marble, '''Python “extend” for a dictionary''', Stackoverflow, 2013 March 23, https://stackoverflow.com/questions/577234/python-extend-for-a-dictionary
s_d.update({I_m4: 4140e6*(1e-3**4)})

#### 하중<br>Load

In [ ]:
V_N = sy.Symbol('V[N]', real=True)

s_d[V_N] = 450e3

#### 리벳<br>Rivet

리벳 지름<br>Rivet diameter

In [ ]:
d_rivet_m = sy.Symbol('d_{rivet}[m]', positive=True)

s_d[d_rivet_m] = 19e-3

리벳 간격<br>Rivet Pitch

In [ ]:
e_rivet_m = sy.Symbol('e_{rivet}[m]', positive=True)

#### 재료<br>Material properties

리벳의 허용 전단응력, 웨브의 허용 지압응력은 다음과 같다.<br>Allowable shear stress of rivet, allowable bearing stress of web are as follows.

In [ ]:
tau_w_Pa, sigma_w_Pa = sy.symbols('tau_{w}[Pa], sigma_{w}[Pa]', real=True)

s_d.update({
    tau_w_Pa: 100e6, 
    sigma_w_Pa: 280e6,
})

### 앵글에 가해지는 전단력<br>Shear force on an angle

#### 앵글의 $Q$<br>$Q$ of an angle

In [ ]:
Q_m3 = A_angle_m2 * y_bar_m
Q_m3

#### 앵글의 전단력<br>Shear force of an angle

p. 220 eq. 5.10a

$$
F_{angle}=\frac{V\cdot Q_{angle}\cdot e_{rivet}}{I}
$$

In [ ]:
F_angle_N = V_N * Q_m3 * e_rivet_m / I_m4
F_angle_N

### 리벳 한 개로 버틸 수 있는 전단력<br>Shear force that one rivet can endure

#### 리벳의 단면적<br>Cross section area of a rivet

In [ ]:
A_rivet_m2 = sy.pi * (d_rivet_m/2) **2

#### 리벳 한 개로 버틸 수 있는 전단력<br>Shear force that one rivet can endure

허용 전단응력에 리벳의 단면적을 곱한다.<br>Multiply allowable shear stress with cross section area of the rivet

In [ ]:
F_w_N = tau_w_Pa * A_rivet_m2
F_w_N

결과값을 확인해 본다.<br>Check the result value

In [ ]:
float(F_w_N.subs(s_d))

### 허용 지압응력으로 리벳 구멍 하나가 버틸 수 있는 힘<br>Force that a rivet hole can endure, considering allowable bearing stress.

#### 리벳 구멍의 단면적<br>Projected area of a rivet hole

In [ ]:
A_bearing_m2 = d_rivet_m * t_web_m
A_bearing_m2

허용응력에 리벳 구멍 벽의 단면적을 곱한다.<br>Multiply this area to the allowable bearing stress.

In [ ]:
F_bearing_w_N = sigma_w_Pa * A_bearing_m2
F_bearing_w_N

힘의 N 값은 다음과 같다.<br>Value of the force in N is as follows.

In [ ]:
float(F_bearing_w_N.subs(s_d))

### 리벳 간격<br>Rivet pitch

#### 전단력 기준<br>Based on shear force

위에서 구한 전단력과 허용 하중을 등식으로 만든다. 리벳이 이중 전단 상태로 전단면이 두개이므로 허용 하중 쪽에 2를 곱한다.<br>Equate the shear force and allowed load. Because reviet has two shear planes, multiply 2 to allowable force side.

In [ ]:
tau_eq = sy.Eq(F_angle_N, F_w_N * 2)
tau_eq

이 식을 푼 결과로 다음과 같이 리벳 간격을 얻는다.<br>Solving this equation gives rivet pitch as follows.

In [ ]:
tau_e_m_sol_list = sy.solve(tau_eq, e_rivet_m)
tau_e_m_sol_list

m 단위 값으로는 다음과 같다.<br>Value in m unit is as follows.

In [ ]:
tau_e_m_sol = float(tau_e_m_sol_list[0].subs(s_d))
tau_e_m_sol

mm 단위로는 다음과 같다.<br>Value in mm unit is as follows.

In [ ]:
tau_e_mm_sol = tau_e_m_sol * 1e3
tau_e_mm_sol

#### 지압력 기준<br>Based on bearing force

다음 질문을 한번 생각해 보자. Let's think about following questions.

* 왜 아래 식에서 $F_{angle}$ 에 2가 곱해지는가? In the following question, why two is multiplied to $F_{angle}?$
* 이 지압력은 웨브 기준인가? 아니면 앵글 기준인가? Is this bearing force on the web or on the angle?
* 다른 부분의 지압력을 계산할 필요가 있는가? 그 까닭은? Do we need to calculate the bearing force on the other side? Why?

In [ ]:
sigma_eq = sy.Eq(F_angle_N * 2, F_bearing_w_N)
sigma_eq

리벳 간격<br>Rivet pitch

In [ ]:
sigma_e_m_sol_list = sy.solve(sigma_eq, e_rivet_m)
sigma_e_m_sol_list

m 값<br>Value in m

In [ ]:
sigma_e_m_sol = float(sigma_e_m_sol_list[0].subs(s_d))
sigma_e_m_sol

mm 값<br>Value in mm

In [ ]:
sigma_e_mm_sol = sigma_e_m_sol * 1e3
sigma_e_mm_sol